In [7]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt
# from scipy.ndimage import label


#%pip install opencv-python # Required to import cv2 !!!
# import cv2

from IPython.display import display, Latex, Math

import os
import time

plt.rcParams['figure.dpi'] = 250

# Task 1

In [ ]:
A_1 = np.array([
    [1, 0],
    [0, 1]
]) 

A_2 = np.array([
    [1, 2],
    [1, 1], 
    [1, 2]
])

A_3 = np.array([
    [2, 1, 1],
    [2, 1, 1], 
    [1, 1, 2]
])

A_4 = np.array([
    [2, 1, 0],
    [1, 2, 3], 
    [0, 3, 3]
])